In [1]:
import os
import numpy
import re
import random
from scipy.stats import gaussian_kde, pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.transforms as mtransforms
import math
import matplotlib.pyplot as plt

In [2]:
WORK_SPACE = os.getcwd()

PIXEL_PAIRS_MAX = 500

MONTH_LABEL = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

MONTH_SOUTH2NORTH = {    # south -> north
    '01': '07',
    '02': '08',
    '03': '09',
    '04': '10',
    '05': '11',
    '06': '12',
    '07': '01',
    '08': '02',
    '09': '03',
    '10': '04',
    '11': '05',
    '12': '06'
}


def identifer(data):
    down,up = numpy.nanpercentile(data,[0,75])
    IQR = up-down
    lower_limit = down - 1.5*IQR
    upper_limit = up + 1.5*IQR
    result = numpy.where(data > upper_limit,numpy.nan, data)
    result = numpy.where(result < lower_limit,numpy.nan, result)
    return result


def add_right_cax(ax, pad, width):

    axpos = ax.get_position()
    caxpos = mtransforms.Bbox.from_extents(axpos.x1 + pad, axpos.y0, axpos.x1 + pad + width, axpos.y1)
    cax = ax.figure.add_axes(caxpos)

    return cax


def mapping_scatter(Y, X, figure_title='demo', band_name='band3', axis_min=0.0, axis_max=0.5, save_flag=0):
    # filter    
#     if band_name == 'band3':
#         axis_max = 0.3
    
    lim_x = numpy.copy(X)
    lim_y = numpy.copy(Y)

    up4_x = numpy.quantile(lim_x, 0.98, interpolation='higher')
    up4_y = numpy.quantile(lim_y, 0.98, interpolation='higher')
    lim_x[lim_x > up4_x] = numpy.nan
    lim_y[lim_y > up4_y] = numpy.nan

    lim_mask = (lim_x*lim_y)*0+1
    temp_x = lim_x*lim_mask
    temp_y = lim_y*lim_mask
    temp_x = temp_x[~numpy.isnan(temp_x)]
    temp_y = temp_y[~numpy.isnan(temp_y)]
    
    diff_array = abs(numpy.array(temp_y)-numpy.array(temp_x))/abs(numpy.minimum(numpy.array(temp_x), numpy.array(temp_y)))
    diff_array_filtered = numpy.array(identifer(diff_array))
    show_x = (diff_array_filtered*0+1)*temp_x
    show_y = (diff_array_filtered*0+1)*temp_y
    
    x_o = show_x[~numpy.isnan(show_x)]
    y_o = show_y[~numpy.isnan(show_y)]

    fig = plt.figure(figsize=(4, 4))
    ax1 = fig.add_subplot(111, aspect='equal',facecolor='whitesmoke', alpha=0.1)
    ax1.grid(linestyle='--', linewidth=0.3)

    model = LinearRegression()
    x = x_o.reshape(-1, 1)
    model.fit(x, y_o)
    y_pred = model.predict(x)
    xx = numpy.arange(axis_min, axis_max + 0.1, 0.05)
    k = model.coef_[0]
    b = model.intercept_
    yy = k * xx + b
    
    N = len(x_o)
    x = numpy.arange(axis_min, axis_max + 1)
    y = 1 * x

#     g_x, g_y = numpy.mgrid[axis_min:axis_max:500j, axis_min:axis_max:500j]
#     positions = numpy.vstack([g_x.ravel(), g_y.ravel()])
#     values = numpy.vstack([X, Y])
#     kernel = gaussian_kde(values)
#     Z = numpy.reshape(kernel(positions).T, g_x.shape)
    
    # Calculate the point density
    xy = numpy.vstack([x_o, y_o])
    z = gaussian_kde(xy)(xy)
    idx = z.argsort()
    x_, y_, z = x_o[idx], y_o[idx], z[idx]
    z = (z-numpy.min(z))/(numpy.max(z)-numpy.min(z))*100
    im = ax1.scatter(x_, y_, marker='o', c=z, s=8, cmap='turbo')
    
    # Calculate parameters
    r2 = r2_score(y_o, x_o)
    rmse = numpy.sqrt(mean_squared_error(y_o, x_o))
    rmse_r = (rmse/numpy.mean(x_o))*100 # %
    bias = numpy.mean(y_o-x_o)
    bias_r = (bias/numpy.mean(x_o))*100 # %
    r_, p = pearsonr(x_o, y_o)
    p_str = '%.3e' % p
    
    print('R²:', r2)
    print('relative RMSE (%):', rmse_r)
    print('RMSE:', rmse)
    print('relative Bias (%):', bias_r)
    print('Bias:', bias)
    print('r_:', bias)    

    ax1.minorticks_on()
    x_major_locator = plt.MultipleLocator(0.1)
    x_minor_locator = plt.MultipleLocator(0.05)
    ticks_interval = 0.1
    if band_name == 'band4':
        x_major_locator = plt.MultipleLocator(0.2)
        x_minor_locator = plt.MultipleLocator(0.1)
        ticks_interval = 0.2
    ax1.xaxis.set_minor_locator(x_minor_locator)
    ax1.xaxis.set_major_locator(x_major_locator)
    ax1.yaxis.set_minor_locator(x_minor_locator)
    ax1.yaxis.set_major_locator(x_major_locator)
    ax1.set_xticks(numpy.arange(axis_min, axis_max + ticks_interval, ticks_interval))
    ax1.set_yticks(numpy.arange(axis_min+ticks_interval, axis_max-ticks_interval, ticks_interval))

    ax1.tick_params(axis="y", which='minor', length=3, direction='in', labelsize=15)
    ax1.tick_params(axis="y", which='major', length=5, direction='in', labelsize=15)

    ax1.tick_params(axis="x", which='minor', length=3, direction='in', labelsize=15)
    ax1.tick_params(axis="x", which='major', length=5, direction='in', labelsize=15)

    # ax1.spines['right'].set_color('none')
    # ax1.spines['top'].set_color('none')
    ax1.spines['left'].set_linewidth(1)
    ax1.spines['bottom'].set_linewidth(1)


    band_label = {
        'band3': 'Band3',
        'band4': 'Band4',
    }

    ax1.set_ylabel("AHI TOA reflectance", fontsize=15)
    ax1.set_xlabel("MISR TOA reflectance", fontsize=15)

#     ax1.imshow(numpy.rot90(Z), cmap=plt.cm.gist_earth_r, extent=[axis_min, axis_max, axis_min, axis_max], alpha=0.8, zorder=0)
#     ax1.plot(X, Y, 'k.', markersize=0.5, alpha=0.8, zorder=4)
    ax1.plot(x, y, color='k', linewidth=1, linestyle='-', zorder=1)
    ax1.plot(xx, yy, color='r', linewidth=1, linestyle='-.', zorder=2)
    
#     label_str = 'y = {}x + {}\nRMSE = {}\nr = {}\n'.format(round(k, 2), round(b, 2), round(rmse, 3), round(r_, 2))
#     if b < 0:
#         label_str = 'y = {}x - {}\nRMSE = {}\nr = {}\n'.format(round(k, 2), abs(round(b, 2)), round(rmse, 3), round(r_, 2))

#     text_x = axis_min + (axis_max - axis_min) * 0.07
#     text_y = axis_max - (axis_max - axis_min) * 0.35
#     ax1.text(text_x, text_y, s=label_str, fontsize=18)
    
#     band_label = band_label[band_name]
#     text_x2 = axis_min + (axis_max - axis_min) * 0.7
#     text_y2 = axis_min + (axis_max - axis_min) * 0.1
#     if band_name == 'band3':
#         ax1.text(text_x2, text_y2, color='red', s=band_label, fontsize=18)
#     else:
#         ax1.text(text_x2, text_y2, color='firebrick', s=band_label, fontsize=18)
    
    label_str = 'y = {}x + {}\nRRMSE: {}%\nRbias: {}%\nr: {}'.format(round(k, 2), round(b, 2), round(rmse_r, 2), round(bias_r, 2),round(r_, 2))
    if b < 0:
        label_str = 'y = {}x + {}\nRRMSE: {}%\nRbias: {}%\nr: {}'.format(round(k, 2), round(b, 2), round(rmse_r, 2), round(bias_r, 2),round(r_, 2))

    text_x = axis_min+(axis_max - axis_min)*0.024
    text_y = axis_max-(axis_max - axis_min)*0.368
    ax1.text(text_x, text_y, s=label_str, fontsize=18)
            
    label_str2 = 'R²: {}\nbias: {}\nRMSE: {}\n'.format(round(r2, 2), round(bias, 3), round(rmse, 3))
    text_x2 = axis_max-(axis_max - axis_min)*0.02
    text_y2 = axis_min-(axis_max - axis_min)*0.06
    ax1.text(text_x2, text_y2, s=label_str2, fontsize=18, horizontalalignment='right')

    cax = add_right_cax(ax1, pad=0.01, width=0.03)
    cb = fig.colorbar(im, cax=cax)
    
    ax1.set_xlim(axis_min, axis_max)
    ax1.set_ylim(axis_min, axis_max)

    mapping_folder = os.path.join(WORK_SPACE, 'month_scatter_LC_north_nadir')
    figure_folder = os.path.join(mapping_folder, str(PIXEL_PAIRS_MAX))
    if not os.path.exists(figure_folder):
        os.makedirs(figure_folder)
    fig_filename = os.path.join(figure_folder, figure_title + '.png')
    fig.savefig(fig_filename, dpi=1000, bbox_inches='tight')
    print(fig_filename)
    plt.close(fig)
    plt.clf()

    # plt.show()
    # slope r RMSE
    return k, r_, rmse


if __name__ == "__main__":

    # folder_l1_list = ['0', '26', '45', '60', '70']
    folder_l1_list = ['45']
    folder_l2_list = ['0', '1']
    lc_type = ['12', '4']

    slope_list = []
    slope_labels = []
    for folder_l1 in folder_l1_list:
        folder_l1_path = os.path.join(WORK_SPACE, folder_l1)
        for folder_l2_idx in range(len(folder_l2_list)):
            folder_l2 = folder_l2_list[folder_l2_idx]
            month_slope_b3 = numpy.zeros((12,))
            month_r_b3 = numpy.zeros((12,))
            month_rmse_b3 = numpy.zeros((12,))
            month_slope_b4 = numpy.zeros((12,))
            month_r_b4 = numpy.zeros((12,))
            month_rmse_b4 = numpy.zeros((12,))
            for month_idx in range(len(MONTH_LABEL)):
                month = MONTH_LABEL[month_idx]
                # each png
                misr_SR_band3_item_list = []
                ahi_SR_band3_item_list = []
                misr_SR_band4_item_list = []
                ahi_SR_band4_item_list = []
                folder_l2_path = os.path.join(folder_l1_path, folder_l2)
                roi_folder_list = os.listdir(folder_l2_path)
                for roi_folder in roi_folder_list:
                    roi_infos = roi_folder.split('_')
                    roi_lc = roi_infos[1]
                    obj_lc = lc_type[folder_l2_idx]
                    if roi_lc == obj_lc:
                        roi_lat = float(roi_infos[2])                    
                        is_south_roi = 0
                        if roi_lat < 0:
                            is_south_roi = 1
                        if not is_south_roi:    # only north
                            roi_folder_path = os.path.join(folder_l2_path, roi_folder)
                            roi_file_list = os.listdir(roi_folder_path)
                            roi_misr_SR_band3_list = []
                            roi_ahi_SR_band3_list = []
                            roi_misr_SR_band4_list = []
                            roi_ahi_SR_band4_list = []
                            for roi_file in roi_file_list:
                                matchObj = re.search(r'(\d+)_band(\d+)_(\d+).npy', str(roi_file))
                                if matchObj:
                                    ahi_time_str = matchObj.group(1)
                                    band_str = matchObj.group(2)
                                    # camera_idx_str = matchObj.group(3)

                                    obs_month = ahi_time_str[4:6]
                                    if is_south_roi:
                                        obs_month = MONTH_SOUTH2NORTH[obs_month]
                                    obs_month_idx = int(obs_month) - 1

                                    if obs_month_idx == month_idx:
                                        SR_npy_path = os.path.join(roi_folder_path, roi_file)
                                        ROI_SR_pair = numpy.load(SR_npy_path, allow_pickle=True)[0]
                                        misr_sr = ROI_SR_pair['misr_toa']
                                        ahi_sr = ROI_SR_pair['ahi_toa2misr']
                                        x_3Darray_np_1d = misr_sr.flatten()
                                        x_3Darray_np_1d = x_3Darray_np_1d[~numpy.isnan(x_3Darray_np_1d)]
                                        y_3Darray_np_1d = ahi_sr.flatten()
                                        y_3Darray_np_1d = y_3Darray_np_1d[~numpy.isnan(y_3Darray_np_1d)]
                                        if band_str == '3':
                                            roi_misr_SR_band3_list.extend(x_3Darray_np_1d)
                                            roi_ahi_SR_band3_list.extend(y_3Darray_np_1d)
                                        if band_str == '4':
                                            roi_misr_SR_band4_list.extend(x_3Darray_np_1d)
                                            roi_ahi_SR_band4_list.extend(y_3Darray_np_1d)
                                    # keep pixel count same
                                    if len(roi_misr_SR_band3_list) == len(roi_misr_SR_band4_list):
                                        misr_SR_band3_item_list.extend(roi_misr_SR_band3_list)
                                        ahi_SR_band3_item_list.extend(roi_ahi_SR_band3_list)
                                        misr_SR_band4_item_list.extend(roi_misr_SR_band4_list)
                                        ahi_SR_band4_item_list.extend(roi_ahi_SR_band4_list)

                print('Random NO.:', PIXEL_PAIRS_MAX)
                print(folder_l1, folder_l2)
                print('MISR_SR_Band3_NO.', 'AHI_SR_Band3_NO.', 'MISR_SR_Band4_NO.', 'AHI_SR_Band4_NO.')
                print(len(misr_SR_band3_item_list), len(ahi_SR_band3_item_list), len(misr_SR_band4_item_list), len(ahi_SR_band4_item_list))

                if len(misr_SR_band3_item_list) > PIXEL_PAIRS_MAX:
                    # random pairs mapping
                    index_array = random.sample([idx for idx in range(len(misr_SR_band3_item_list))], PIXEL_PAIRS_MAX)
                    index_array = numpy.sort(index_array).tolist()

                    misr_SR_band3_pts = numpy.array(misr_SR_band3_item_list)
                    show_misr_sr_b3 = misr_SR_band3_pts[index_array]
                    ahi_SR_band3_pts = numpy.array(ahi_SR_band3_item_list)
                    show_ahi_sr_b3 = ahi_SR_band3_pts[index_array]
                    figure_title = folder_l1 + '_' + folder_l2 + '_b3' + '_' + str(month_idx) + month + '_' + str(PIXEL_PAIRS_MAX)
                    slope_b3, r_b3, rmse_b3 = mapping_scatter(show_ahi_sr_b3, show_misr_sr_b3, figure_title, 'band3', axis_min=0.0, axis_max=0.5)
                    month_slope_b3[month_idx] = round(slope_b3, 2)
                    month_r_b3[month_idx] = round(r_b3, 2)
                    month_rmse_b3[month_idx] = round(rmse_b3, 3)

                    misr_SR_band4_pts = numpy.array(misr_SR_band4_item_list)
                    show_misr_sr_b4 = misr_SR_band4_pts[index_array]
                    ahi_SR_band4_pts = numpy.array(ahi_SR_band4_item_list)
                    show_ahi_sr_b4 = ahi_SR_band4_pts[index_array]
                    figure_title = folder_l1 + '_' + folder_l2 + '_b4' + '_' + str(month_idx) + month + '_' + str(PIXEL_PAIRS_MAX)
                    slope_b4, r_b4, rmse_b4 = mapping_scatter(show_ahi_sr_b4, show_misr_sr_b4, figure_title, 'band4', axis_min=0.0, axis_max=0.8)
                    month_slope_b4[month_idx] = round(slope_b4, 2)
                    month_r_b4[month_idx] = round(r_b4, 2)
                    month_rmse_b4[month_idx] = round(rmse_b4, 3)
                
                else:
                    # all pairs mapping
                    pairs_no = len(misr_SR_band3_item_list)
                    if pairs_no > 3:

                        misr_SR_band3_pts = numpy.array(misr_SR_band3_item_list)
                        ahi_SR_band3_pts = numpy.array(ahi_SR_band3_item_list)
                        figure_title = folder_l1 + '_' + folder_l2 + '_b3' + '_' + str(month_idx) + month + '_' + str(pairs_no)
                        slope_b3, r_b3, rmse_b3 = mapping_scatter(ahi_SR_band3_pts, misr_SR_band3_pts, figure_title, 'band3', axis_min=0.0, axis_max=0.5)
                        month_slope_b3[month_idx] = round(slope_b3, 2)
                        month_r_b3[month_idx] = round(r_b3, 2)
                        month_rmse_b3[month_idx] = round(rmse_b3, 3)

                        misr_SR_band4_pts = numpy.array(misr_SR_band4_item_list)
                        ahi_SR_band4_pts = numpy.array(ahi_SR_band4_item_list)
                        figure_title = folder_l1 + '_' + folder_l2 + '_b4' + '_' + str(month_idx) + month + '_' + str(pairs_no)
                        slope_b4, r_b4, rmse_b4 = mapping_scatter(ahi_SR_band4_pts, misr_SR_band4_pts, figure_title, 'band4', axis_min=0.0, axis_max=0.8)
                        month_slope_b4[month_idx] = round(slope_b4, 2)
                        month_r_b4[month_idx] = round(r_b4, 2)
                        month_rmse_b4[month_idx] = round(rmse_b4, 3)
            print('Slope, r, RMSE')
            print(month_slope_b3)
            slope_list.append(month_slope_b3)
            slope_labels.append(folder_l1 + '_' + folder_l2 + '_band3')
            print(month_r_b3)
            print(month_rmse_b3)
            print(month_slope_b4)
            slope_list.append(month_slope_b4)
            slope_labels.append(folder_l1 + '_' + folder_l2 + '_band4')
            print(month_r_b4)
            print(month_rmse_b4)

Random NO.: 500
45 0
MISR_SR_Band3_NO. AHI_SR_Band3_NO. MISR_SR_Band4_NO. AHI_SR_Band4_NO.
513 513 513 513
R²: 0.6618003070195859
relative RMSE (%): 29.077240411863315
RMSE: 0.10896072144962321
relative Bias (%): 25.52787252448115
Bias: 0.09566022662200892
r_: 0.09566022662200892
/disk1/workspace/20231012/01_Ray_nadir/month_scatter_LC_north_nadir/500/45_0_b3_0Jan_500.png
R²: 0.279636449309279
relative RMSE (%): 38.0642438867538
RMSE: 0.1640009916468996
relative Bias (%): 35.63333268865423
Bias: 0.1535273343143125
r_: 0.1535273343143125
/disk1/workspace/20231012/01_Ray_nadir/month_scatter_LC_north_nadir/500/45_0_b4_0Jan_500.png
Random NO.: 500
45 0
MISR_SR_Band3_NO. AHI_SR_Band3_NO. MISR_SR_Band4_NO. AHI_SR_Band4_NO.
162 162 162 162
R²: -0.024920179456881852
relative RMSE (%): 22.644570692169857
RMSE: 0.1157456457280299
relative Bias (%): 19.027650832593345
Bias: 0.09725809167437859
r_: 0.09725809167437859
/disk1/workspace/20231012/01_Ray_nadir/month_scatter_LC_north_nadir/500/45_0_b3_1

/disk1/workspace/20231012/01_Ray_nadir/month_scatter_LC_north_nadir/500/45_1_b3_0Jan_500.png
R²: -1.9808807197669105
relative RMSE (%): 68.75653302401092
RMSE: 0.1675129814998365
relative Bias (%): 65.03828362255338
Bias: 0.1584541326049886
r_: 0.1584541326049886
/disk1/workspace/20231012/01_Ray_nadir/month_scatter_LC_north_nadir/500/45_1_b4_0Jan_500.png
Random NO.: 500
45 1
MISR_SR_Band3_NO. AHI_SR_Band3_NO. MISR_SR_Band4_NO. AHI_SR_Band4_NO.
1062 1062 1062 1062
R²: 0.06476195635439674
relative RMSE (%): 46.32086231543545
RMSE: 0.11983421572647086
relative Bias (%): 37.660189820453645
Bias: 0.09742865494410775
r_: 0.09742865494410775
/disk1/workspace/20231012/01_Ray_nadir/month_scatter_LC_north_nadir/500/45_1_b3_1Feb_500.png
R²: -2.7626725165573633
relative RMSE (%): 57.516549428155315
RMSE: 0.20041221529297715
relative Bias (%): 54.35701646544595
Bias: 0.18940305346662809
r_: 0.18940305346662809
/disk1/workspace/20231012/01_Ray_nadir/month_scatter_LC_north_nadir/500/45_1_b4_1Feb_500.

<Figure size 432x288 with 0 Axes>